<a href="https://colab.research.google.com/github/johnsonjzhou/comp90089-project/blob/main/sql/initial_cohort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **COMP90089 Final Project**

## **Selecting Initial Patient Cohort from MIMICIV**

In [3]:
from google.colab import drive

drive.mount('/content/drive')
path = '/content/drive/My Drive'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install -U pandasql

In [5]:
#Set up the environement

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import functools as ft
from pandasql import sqldf
%matplotlib inline


#Project_ID
project_id = "mimic-iv-projects" 

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
    return pd.io.gbq.read_gbq(
        query,
        project_id=project_id,
        dialect='standard')


### **SQL queries**

In [6]:
#Select patients from ICU_stays based on: n_stay >= 1 and average los >= 3.3 per 	hadm_id:

c1= f"""
SELECT
    icustays.subject_id As subject_id,
    icustays.hadm_id As hadm_id,
    count(icustays.stay_id) As n_stays,
    sum(icustays.los) As sum_los
FROM
  `physionet-data.mimiciv_icu.icustays` AS icustays
GROUP BY
    subject_id,	hadm_id
HAVING 
   n_stays >= 1 AND sum_los >= 3.3
ORDER BY
    subject_id,	hadm_id
"""

In [7]:
cohort_c1 = f"""
    
    SELECT c1.subject_id As subject_id,
           c1.hadm_id As hadm_id,
           icustays.stay_id As stay_id,
           c1.n_stays As n_stays,
           c1.sum_los As sum_los
    FROM  ({c1}) As c1
    INNER JOIN `physionet-data.mimiciv_icu.icustays` AS icustays
    ON icustays.hadm_id = c1.hadm_id
"""

In [8]:
#Select patients from cohort_c1 based on the age range (18 - 90) and survival status (dod is null) criteria from Table: hosp.patients :
cohort_q1= f"""
      SELECT cohort_c1.*,
            patients.gender As gender,
            patients.anchor_age As anchor_age
      FROM ({cohort_c1}) As cohort_c1
      INNER JOIN `physionet-data.mimiciv_hosp.patients` As patients
        ON cohort_c1.subject_id = patients.subject_id AND (patients.anchor_age BETWEEN 18 AND 90) AND  (patients.dod IS NULL)
        ORDER BY
          patients.subject_id
        """

In [9]:
#Demographic_1
# Presence of infection
# Table: mimiciv_derived.suspicion_of_infection
# Presence of infection (suspected_infection = 1)


demographic_1 = f"""
    SELECT suspicion_of_infection.subject_id As subject_id,
          suspicion_of_infection.suspected_infection As suspected_infection,
    FROM `physionet-data.mimiciv_derived.suspicion_of_infection` As suspicion_of_infection
"""

In [10]:
#Demographic_2
# Table: mimiciv_hosp.admissions
# Type of admission (admission_type)

#Note: Some patients have more than one admission type

demographic_2 = f"""
    SELECT  DISTINCT admission.subject_id As subject_id,
            admission.hadm_id As hadm_id,
            admission.admission_type As admission_type
    FROM `physionet-data.mimiciv_hosp.admissions` As admission
"""

In [11]:
##Demographic_3
# Table: mimiciv_derived.first_day_weight
# weight
# Table: mimiciv_derived.first_day_height
# height
demographic_3 = f"""
    SELECT DISTINCT first_day_weight.subject_id As subject_id,
            first_day_weight.stay_id As stay_id,
            first_day_weight.weight As weight,
            first_day_height.height As height,
    FROM `physionet-data.mimiciv_derived.first_day_weight` as first_day_weight
    LEFT JOIN `physionet-data.mimiciv_derived.first_day_height` As first_day_height
    ON first_day_weight.stay_id = first_day_height.stay_id
   """

In [12]:
#Cohort_Demographic
demographic = f"""
    SELECT DISTINCT demographic_2.*, 
                    demographic_1.suspected_infection,
                    demographic_3.weight,
                    demographic_3.stay_id,
                    demographic_3.height
   FROM ({demographic_2}) As  demographic_2
   LEFT JOIN ({demographic_1}) As  demographic_1
     ON demographic_2.subject_id = demographic_1.subject_id
   LEFT JOIN ({demographic_3}) As  demographic_3
     ON demographic_1.subject_id = demographic_3.subject_id
    """

In [13]:
#Cohort_Demographic

cohort_q2 = run_query(f"""
        SELECT DISTINCT cohort_q1.*,
                        demographic.admission_type,
                        demographic.suspected_infection,
                        demographic.weight,
                        demographic.height
        FROM ({cohort_q1}) As cohort_q1
        LEFT JOIN ({demographic}) As demographic
         ON cohort_q1.hadm_id = demographic.hadm_id
        """)

In [14]:
cohort_q2.to_csv('/content/drive/MyDrive/cohort_criteria_data.csv')

In [15]:
# Vital signs
#Table: mimiciv_derived.first_day_vitalsign`

#heart rate(heart_rate_mean)
#respiratory_rate (resp_rate_mean)
#body_temp (temperature_mean)

# Invasive mean arterial pressure (mbp_mean) 
# Invasive systolic blood pressure (sbp_mean) (min and max)
# Invasive diastolic blood pressure (dbp_mean) (min and max)
# Oxygen saturation (SPO2_mean)

#(Non-invasive values) not included in the first_day_vitalsign
# Non-invasive mean arterial pressure
# Non-invasive Systolic blood pressure
# Non-invasive Diastolic blood pressure

cohort_vitalsigns = run_query(f"""
SELECT DISTINCT 
       first_day_vitalsigns.subject_id As subject_id,
       first_day_vitalsigns.stay_id As stay_id,
       first_day_vitalsigns.heart_rate_mean As heart_rate_mean,
       first_day_vitalsigns.resp_rate_mean As resp_rate_mean,
       first_day_vitalsigns.temperature_mean As temperature_mean,
       first_day_vitalsigns.mbp_mean As mbp_mean,
       first_day_vitalsigns.sbp_mean As sbp_mean,
       first_day_vitalsigns.dbp_mean As dbp_mean,
       first_day_vitalsigns.spo2_mean As spo2_mean
 FROM `physionet-data.mimiciv_derived.first_day_vitalsign`  As first_day_vitalsigns 
""")

In [16]:
cohort_vitalsigns.to_csv('/content/drive/MyDrive/cohort_vitalsigns.csv')

In [17]:

# Laboratory tests


#Table: mimiciv_derived.first_day_lab

# Albumin(albumin_min, albumin_max)
# Blood urea nitrogen (bun_min, bun_max)
# Calcium (calcium_min, calcium_max)
# creatinine (creatinine_min, creatinine_max)
# Glucose (glucose_min, glucose_max)
# Bicarbonate (bicarbonate_min, bicarbonate_max)
# Potassium (potassium_min, potassium_max)
# Sodium (sodium_min, sodium_max)
# Platelets (platelets_min, platelets_max)
# Bilirubin (bilirubin_total_min, bilirubin_total_max)
# White blood cell count (wbc_min, wbc_max)

#Table: mimiciv_derived.first_day_bg

# Lactate (lactate_min, lactate_max)
# pH (ph_min, ph_max)
# Partial pressure of carbon dioxide (pco2_min, pco2_max)
# Partial pressure of Oxygen (po2_min, po2_max)
# PaO2/FiO2 ratio (pao2fio2ratio_min, pao2fio2ratio_max)

#Not Found! 

# Lactate Dehydrogenase,
# Magnesium (Mg), 
#Leukocytes
# Urea

cohort_lab_tests_1 = run_query(f"""
SELECT DISTINCT 
       lab_tests.subject_id As subject_id,
       lab_tests.stay_id As stay_id,
       lab_tests.albumin_min As albumin_min,
       lab_tests.albumin_max As albumin_max,
       lab_tests.bun_min As bun_min,
       lab_tests.bun_max As bun_max,
       lab_tests.calcium_min As calcium_min,
       lab_tests.calcium_max As calcium_max,
       lab_tests.creatinine_min As creatinine_min,
       lab_tests.creatinine_max As creatinine_max,
       lab_tests.glucose_min As glucose_min,
       lab_tests.glucose_max As glucose_max,
       lab_tests.bicarbonate_min As bicarbonate_min,
       lab_tests.bicarbonate_max As bicarbonate_max,
       lab_tests.potassium_min As potassium_min,
       lab_tests.potassium_max As potassium_max,
       lab_tests.sodium_min As sodium_min,
       lab_tests.sodium_max As sodium_max,
       lab_tests.platelets_min As platelets_min,
       lab_tests.platelets_max As platelets_max,
       lab_tests.bilirubin_total_min As bilirubin_total_min,
       lab_tests.bilirubin_total_max As bilirubin_total_max,
       lab_tests.wbc_min As wbc_min,
       lab_tests.wbc_max As wbc_max,
 FROM `physionet-data.mimiciv_derived.first_day_lab`  As lab_tests
""")


In [18]:
cohort_lab_tests_1.to_csv('/content/drive/MyDrive/cohort_lab_tests_1.csv')

In [19]:

       
cohort_lab_tests_2 = run_query(f"""
SELECT DISTINCT 
       lab_bg_tests.subject_id As subject_id,
       lab_bg_tests.stay_id As stay_id,
       lab_bg_tests.lactate_min As lactate_min,
       lab_bg_tests.lactate_max As lactate_max,
       lab_bg_tests.ph_min As ph_min,
       lab_bg_tests.ph_max As ph_max,
       lab_bg_tests.pco2_min As pco2_min,
       lab_bg_tests.pco2_max As pco2_max,
       lab_bg_tests.po2_min As po2_min,
       lab_bg_tests.po2_max As po2_max,
       lab_bg_tests.pao2fio2ratio_min As pao2fio2ratio_min,
       lab_bg_tests.pao2fio2ratio_max As pao2fio2ratio_max
       FROM `physionet-data.mimiciv_derived.first_day_bg` As lab_bg_tests
       """)


In [20]:
cohort_lab_tests_2.to_csv('/content/drive/MyDrive/cohort_lab_tests_2.csv')

In [21]:
# Comorbidities

# Table: mimiciv_derived.charlson
# Charlson index (charlson_comorbidity_index)


cohort_comorbidities_1 = f"""
SELECT DISTINCT 
       charlson_comorbidity.subject_id As subject_id,
       charlson_comorbidity.charlson_comorbidity_index As charlson
FROM `physionet-data.mimiciv_derived.charlson` As charlson_comorbidity
"""

In [22]:
# Table: mimiciv_derived.gcs
# Glasgow coma scale (gcs)

# Table: mimiciv_derived.first_day_sofa
# Sequential Organ Failure Assessment score (SOFA)

cohort_comorbidities_2 = f"""
SELECT DISTINCT 
       gcs_first_day.subject_id As subject_id,
       gcs_first_day.stay_id As stay_id,
       gcs_first_day.gcs_min As gcs,
       first_day_sofa.SOFA As sofa

FROM `physionet-data.mimiciv_derived.first_day_gcs` As gcs_first_day
LEFT JOIN `physionet-data.mimiciv_derived.first_day_sofa` As first_day_sofa
ON gcs_first_day.stay_id = first_day_sofa.stay_id
"""

In [23]:
cohort_comorbidities = run_query(f"""
SELECT DISTINCT 
       cohort_comorbidities_2.*,
       cohort_comorbidities_1.charlson
FROM ({cohort_comorbidities_2}) as cohort_comorbidities_2
LEFT JOIN ({cohort_comorbidities_1}) As cohort_comorbidities_1
ON cohort_comorbidities_2.subject_id = cohort_comorbidities_1.subject_id 
"""   
)

In [24]:
cohort_comorbidities.to_csv('/content/drive/MyDrive/cohort_comorbidities.csv')

In [25]:
# Device use

# Table: mimiciv_derived.ventilation
# Ventilation status (ventilation_status)

cohort_device_use = run_query(f"""
SELECT DISTINCT 
       cohort_q1.*,
       ventilation.ventilation_status As ventilation_status
FROM ({cohort_q1}) As cohort_q1
LEFT JOIN `physionet-data.mimiciv_derived.ventilation` As ventilation
ON ventilation.stay_id = cohort_q1.stay_id
""")

In [26]:
cohort_device_use.to_csv('/content/drive/MyDrive/cohort_device_use.csv')

In [27]:
# Input/output: 


# Table: mimiciv_derived.first_day_urine_output 
# Urine output (urineoutput)


cohort_urine_output = run_query(f"""
SELECT urine_output.subject_id As subject_id,
       urine_output.stay_id As stay_id,
       urine_output.urineoutput As urineoutput,           
FROM `physionet-data.mimiciv_derived.first_day_urine_output` As urine_output
""")

In [28]:
cohort_urine_output.to_csv('/content/drive/MyDrive/cohort_urine_output.csv')

In [29]:
# Input/output: 

# Table: mimiciv_derived.vasopressin 
# Use of vasopressors (vaso_rate)

cohort_vasopressin = run_query(f"""
SELECT DISTINCT
       vasopressin.stay_id As stay_id,
       vasopressin.vaso_rate As vaso_rate
FROM `physionet-data.mimiciv_derived.vasopressin` As vasopressin    
""")


In [30]:
cohort_vasopressin.to_csv('/content/drive/MyDrive/cohort_vasopressin.csv')

### **Join all result tables**

In [31]:
#Join tables: 

cohort_criteria = pd.read_csv('/content/drive/MyDrive/cohort_criteria_data.csv')
cohort_vitalsigns = pd.read_csv('/content/drive/MyDrive/cohort_vitalsigns.csv')
cohort_lab_tests_1 = pd.read_csv('/content/drive/MyDrive/cohort_lab_tests_1.csv')
cohort_lab_tests_2 = pd.read_csv('/content/drive/MyDrive/cohort_lab_tests_2.csv')
cohort_comorbidities = pd.read_csv('/content/drive/MyDrive/cohort_comorbidities.csv')
cohort_device_use = pd.read_csv('/content/drive/MyDrive/cohort_device_use.csv')
cohort_urine = pd.read_csv('/content/drive/MyDrive/cohort_urine_output.csv')
cohort_vasopressin = pd.read_csv('/content/drive/MyDrive/cohort_vasopressin.csv')


In [33]:
#1. Join vital signs
query1 = f"""
SELECT DISTINCT cohort_criteria.*,
       cohort_vitalsigns.* 
FROM cohort_criteria  As cohort_criteria
LEFT JOIN cohort_vitalsigns AS cohort_vitalsigns
ON cohort_criteria.stay_id = cohort_vitalsigns.stay_id
""" 

In [35]:
#Run query
vitalsigns_joined  = sqldf(query1, globals())

In [37]:
#Remove duplicate Ids columns 
vitalsigns_joined = vitalsigns_joined.loc[:,~vitalsigns_joined.columns.duplicated()].copy()

In [42]:
#2. Join lab tests 
query2 = f"""
SELECT DISTINCT vitalsigns_joined.*,
       cohort_lab_tests_1.* 
FROM vitalsigns_joined  As vitalsigns_joined
LEFT JOIN cohort_lab_tests_1 AS cohort_lab_tests_1
ON vitalsigns_joined.stay_id = cohort_lab_tests_1.stay_id
""" 

In [44]:
#Run query
lab_tests_1_joined  = sqldf(query2, globals())

In [45]:
#Remove duplicate Ids columns 
lab_tests_1_joined = lab_tests_1_joined.loc[:,~lab_tests_1_joined.columns.duplicated()].copy()

In [46]:
#3. Join lab tests 
query3 = f"""
SELECT DISTINCT lab_tests_1_joined.*,
       cohort_lab_tests_2.* 
FROM lab_tests_1_joined  As lab_tests_1_joined
LEFT JOIN cohort_lab_tests_2 AS cohort_lab_tests_2
ON lab_tests_1_joined.stay_id = cohort_lab_tests_2.stay_id
""" 

In [47]:
#Run query
lab_tests_2_joined  = sqldf(query3, globals())

In [48]:
#Remove duplicate Ids columns 
lab_tests_2_joined = lab_tests_2_joined.loc[:,~lab_tests_2_joined.columns.duplicated()].copy()

In [49]:
#4. Join comorbidities scores 

query4 = f"""
SELECT DISTINCT lab_tests_2_joined.*,
      cohort_comorbidities.* 
FROM lab_tests_2_joined  As lab_tests_2_joined
LEFT JOIN cohort_comorbidities AS cohort_comorbidities
ON lab_tests_2_joined.stay_id = cohort_comorbidities.stay_id
""" 

In [50]:
#Run query
comorbidities_joined  = sqldf(query4, globals())

In [51]:
#Remove duplicate Ids columns 
comorbidities_joined = comorbidities_joined.loc[:,~comorbidities_joined.columns.duplicated()].copy()

In [52]:
#5. Join used devices

query5 = f"""
SELECT DISTINCT comorbidities_joined.*,
      cohort_device_use.* 
FROM comorbidities_joined  As comorbidities_joined
LEFT JOIN cohort_device_use AS cohort_device_use
ON comorbidities_joined.stay_id = cohort_device_use.stay_id
""" 

In [53]:
#Run query
device_use_joined  = sqldf(query5, globals())

In [54]:
#Remove duplicate Ids columns 
device_use_joined = device_use_joined.loc[:,~device_use_joined.columns.duplicated()].copy()

In [55]:
#6. Join urine output

query6 = f"""
SELECT DISTINCT device_use_joined.*,
      cohort_urine.* 
FROM device_use_joined  As device_use_joined
LEFT JOIN cohort_urine AS cohort_urine
ON device_use_joined.stay_id = cohort_urine.stay_id
""" 

In [56]:
#Run query
urine_output_joined  = sqldf(query6, globals())

In [57]:
#Remove duplicate Ids columns 
urine_output_joined = urine_output_joined.loc[:,~urine_output_joined.columns.duplicated()].copy()

In [58]:
#7. Join vasopressin rate

query7 = f"""
SELECT DISTINCT urine_output_joined.*,
      cohort_vasopressin.* 
FROM urine_output_joined  As urine_output_joined
LEFT JOIN cohort_vasopressin AS cohort_vasopressin
ON urine_output_joined.stay_id = cohort_vasopressin.stay_id
""" 

In [59]:
#Run query
vasopressin_rate_joined  = sqldf(query7, globals())

In [60]:
#Remove duplicate Ids columns 
vasopressin_rate_joined = vasopressin_rate_joined.loc[:,~vasopressin_rate_joined.columns.duplicated()].copy()

In [ ]:
vasopressin_rate_joined 

### **Final result table**

In [62]:
intial_cohort = vasopressin_rate_joined.copy()

In [63]:
intial_cohort.to_csv('/content/drive/MyDrive/initial_cohort_final.csv')

In [64]:
intial_cohort.describe()

,Unnamed: 0,subject_id,hadm_id,stay_id,n_stays,sum_los,anchor_age,suspected_infection,weight,height,...,pco2_max,po2_min,po2_max,pao2fio2ratio_min,pao2fio2ratio_max,gcs,sofa,charlson,urineoutput,vaso_rate
count,377054.000000,3.770540e+05,3.770540e+05,3.770540e+05,377054.000000,377054.000000,377054.000000,375243.000000,358617.000000,185604.000000,...,323238.000000,323238.000000,323238.000000,263615.000000,263615.000000,376227.000000,377054.000000,377054.000000,362352.000000,170218.000000
mean,26525.496924,1.472944e+07,2.510484e+07,3.483722e+07,1.878360,16.948428,56.251903,0.543256,88.208047,171.010166,...,52.556327,71.113269,245.937362,150.817766,316.246210,10.948555,8.390541,5.444451,1727.259606,2.285940
std,14893.367103,2.953427e+06,2.832575e+06,2.920474e+06,1.232633,16.329251,14.345493,0.498126,27.116287,10.631526,...,17.386215,44.971794,150.503682,97.403572,169.922055,4.247900,4.638819,2.668817,1365.410989,3.703602
min,0.000000,1.000235e+07,2.000136e+07,3.000015e+07,1.000000,3.300185,18.000000,0.000000,1.000000,122.000000,...,8.000000,4.000000,14.000000,5.714286,29.000000,3.000000,0.000000,0.000000,-1590.000000,0.060000
25%,13592.000000,1.209509e+07,2.279932e+07,3.233927e+07,1.000000,5.846910,49.000000,0.000000,70.850000,163.000000,...,43.000000,39.000000,114.000000,74.000000,212.000000,8.000000,5.000000,4.000000,770.000000,1.201649
50%,26754.000000,1.451340e+07,2.518074e+07,3.480665e+07,1.000000,10.453785,57.000000,1.000000,85.300000,173.000000,...,49.000000,62.000000,214.000000,135.000000,290.000000,13.000000,8.000000,5.000000,1455.000000,2.400000
75%,39766.000000,1.727769e+07,2.743418e+07,3.733031e+07,2.000000,22.044375,66.000000,1.000000,101.000000,178.000000,...,57.000000,90.000000,370.000000,204.000000,385.000000,14.000000,12.000000,7.000000,2335.000000,2.404810
max,51674.000000,1.999944e+07,2.999962e+07,3.999981e+07,7.000000,101.726238,89.000000,1.000000,1251.000000,203.000000,...,199.000000,566.000000,795.000000,1302.500000,2242.857143,15.000000,21.000000,18.000000,30595.000000,295.350006
